In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
PATH_tweets = '../Data/tweets.csv'

In [3]:
tweets = pd.read_csv(
    PATH_tweets,
    usecols=['id', 'screen_name', 'text'], 
    index_col='id', 
    dtype={'screen_name': str, 'text': str}
    )
tweets = tweets.drop_duplicates(subset='text', keep=False)
tweets = tweets.dropna()

In [4]:
tweets.iloc[0].text

'@unveranonaranja @ruidosafest @franciscamusic @giovannaroa @diegosporzio Gracias querida 💗'

In [5]:
def limpiar_retweet(tweet:str):
    if tweet.startswith('RT'):
        tweet = ''.join(tweet.split(': ')[1:])
    return tweet

def limpiar_hashtag(tweet:str):
    tweet = re.sub(r'#\w+', '', tweet)
    return tweet

def limpiar_url(tweet:str):
    tweet = re.sub(r'http\S+', '', tweet)
    return tweet

def limpiar_emoji(tweet:str):
    tweet = re.sub(r'\\x\w+', '', tweet)
    return tweet

def limpiar_puntuacion(tweet:str):
    tweet = re.sub(r'[^\w\s@]', '', tweet)
    return tweet

def limpiar_espacios(tweet:str):
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

def limpiar_mayusculas(tweet:str):
    tweet = tweet.lower()
    return tweet

def remplazar_tildes(tweet:str):
    tweet = tweet.replace('á', 'a')
    tweet = tweet.replace('é', 'e')
    tweet = tweet.replace('í', 'i')
    tweet = tweet.replace('ó', 'o')
    tweet = tweet.replace('ú', 'u')
    return tweet

def limpiar_texto(tweet:str):
    tweet = limpiar_retweet(tweet)
    tweet = limpiar_hashtag(tweet)
    tweet = limpiar_url(tweet)
    tweet = limpiar_emoji(tweet)
    tweet = limpiar_puntuacion(tweet)
    tweet = limpiar_espacios(tweet)
    tweet = limpiar_mayusculas(tweet)
    tweet = remplazar_tildes(tweet)
    tweets = tweet.strip()
    return tweet

tweets['text'] = tweets.loc[:, 'text'].apply(limpiar_texto)

In [6]:
for i in range(10):
    print(tweets['text'].iloc[i].ljust(150) + f'{i}')

@unveranonaranja @ruidosafest @franciscamusic @giovannaroa @diegosporzio gracias querida                                                              0
@lasurivas @elisaloncon @siliconvalle @valdebenitonata rojas vade                                                                                     1
@teanval0207 @izkia @arturozunigaj excelente                                                                                                          2
@mcubillossigall                                                                                                                                      3
@simonlodijo @ruidosafest @franciscamusic @giovannaroa @diegosporzio cami simplemente la mejor                                                        4
@jaime_bassa @solshra sugiero usar telegram los canales son ilimitadas las suscripciones                                                              5
@hernanlarrain @gabrielboric que va a hacer nada solo prestarle ropa para que se siga eq

In [7]:
# Save tweets
tweets.to_parquet('../Data/tweets.parquet')

In [8]:
def shingles(k, words: np.ndarray):
    shingles = []
    for i in range(0, len(words)):
        shingles.append(' '.join(words[i:i+k]))
    return list(set(shingles))

In [9]:
k = 2 # Shingles size
tweets['shingles'] = tweets['text'].str.split().apply(lambda x: np.array(x)).apply(lambda x: shingles(k, x))

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

In [11]:
shings = tweets['shingles'].to_numpy()
mlb = MultiLabelBinarizer(sparse_output=True)
caracteristica = mlb.fit_transform(shings)

In [12]:
# !pip install pyarrow -q

In [13]:
# Save csr matrix
from scipy.sparse import save_npz

# k 2 por los shingles
save_npz(f'../Data/caracteristica_k{k}.npz', caracteristica)